In [4]:
#import cards_sol_3
from cards_sol_3 import *

# What is Poker May Never Die

Today we'll work on our poker skillz a bit more, this time focusing on test driven development. We'll try to create a Player and a (partially working) Game class, and we can do each by focusing on the test part first, then working on meeting those requirements. 

![Theon](../images/theon.jpeg "Theon")

## Import as a Library

When we are creating new classes that define usefule new types of objects, pasting the code into code cells over and over is not a very elegant solution. Instead, we can put the code for our classes into a normal Python (.py) file, then import that just as we do with other libraries. If we do this, using the classes that we've created is basically the same as using Pandas Dataframes or Numpy Arrays. If the python file is in the same directory as our notebook we can just import it by file name, as we've done here. 

The import statement in the block above will import our current card game, just like any other library like Pandas or math. 

### Using Imported Classes and Methods

Above there are two import statements, the <i>import cards</i> that is commented out, and the <i>from cards import *</i> that is being used. These each do the same thing, but in slightly different ways. The first one imports the entire file, and we need to use the dot notation to access the classes and methods that we want. The second one imports everything from the file, directly as its own thing, so we don't need to use the dot notation.

In the first, we are importing "cards", so if we want to access something that is inside of cards, like Deck, we need to use dot notation of cards.Deck to get to it. We can say that cards is the item that is visible to us, and we need to use dot notation to get to anything inside of it. In the second we are importing each class inside of the cards file on its own, so we don't need to use dot notation to get to it. We can think of that * import like a for loop, it'll grab each class and method in the file and import it on its own. There's no difference in the result of these two, we just need to call functions differently depending on which we use. You can see the same thing with imports from public libraries, most notably with sklearn, there are examples of both types of imports in the documentation, tutorials, and examples.


In [5]:
c = Card("Hearts", "King")
print(c)

King of Hearts


In [6]:
d = Deck(populate=True, shuffle=True)
print(d)

0: Ten of Spades
1: Four of Clubs
2: Six of Spades
3: Seven of Diamonds
4: Six of Clubs
5: Eight of Diamonds
6: Queen of Clubs
7: Seven of Hearts
8: Four of Hearts
9: Ace of Hearts
10: Five of Diamonds
11: Ace of Diamonds
12: Ten of Diamonds
13: Nine of Spades
14: King of Hearts
15: Two of Diamonds
16: Two of Clubs
17: Jack of Hearts
18: Two of Spades
19: Nine of Clubs
20: Seven of Spades
21: Jack of Clubs
22: Eight of Spades
23: Jack of Spades
24: Three of Clubs
25: King of Spades
26: Five of Spades
27: Ten of Clubs
28: Six of Diamonds
29: Three of Diamonds
30: Six of Hearts
31: Five of Clubs
32: Queen of Hearts
33: Nine of Diamonds
34: Eight of Clubs
35: Four of Spades
36: King of Diamonds
37: Five of Hearts
38: Jack of Diamonds
39: Ace of Clubs
40: Eight of Hearts
41: Ace of Spades
42: Queen of Diamonds
43: Seven of Clubs
44: Nine of Hearts
45: Four of Diamonds
46: Three of Spades
47: King of Clubs
48: Queen of Spades
49: Two of Hearts
50: Three of Hearts
51: Ten of Hearts



### Warning Note

When things are imported they are "brought into" the current environment in which the program is running. Sometimes if you're changing things, the current definition of one of the classes or functions might still be old in the environment, even if you've updated the code. If there are weird errors, a first step is to restart the kernel (the restart button in VS Code, or close and reopen the file) and rerun the code. There is a possibility that things are just out of sync.

Saving the .py file that you're working on also matters, as the import will grab the file from disk, not the current state of whatever you're working on. If that file is imported up in a cell near the top, that'll also need to be rerun to get the new version of the file.

<b>tl;dr</b> If you're getting weird errors, restart the kernel and rerun the code. If that doesn't work, save the file, close it, reopen it, and rerun the code.

## Do You Want to Play a Game? 

We can expand the capabilities of our poker infrastructure by adding a little more logic and functionality. Right now, we have something that has some of the functionality we need:
<ul>
<li> Cards exist (and are likely complete). </li>
<li> Decks exist and are probably mostly complete. </li>
<li> Hands inheirit from Decks, and are largely complete outside of building out the scoring and comparison logic. </li>
</ul>

Some things that we likely want to add next, along with what we need to consider:
<ul>
<li> A game class, that will act initially as a testing harness to help us, and grow into playing poker. </li>
    <ul>
    <li> This class should "hold" the game. </li>
    <li> Creation of a game should make players, and we should have the ability to play multiple rounds. </li>
    <li> <b>Note:</b> as we go through making the game, we might find things we want to change or move around elsewhere in the code. </li>
    </ul>
<li> A player class, that will represent a player in a poker game. </li>
    <ul>
    <li> This class should hold attributes of the player - name and their bankroll. </li>
    <li> The player will also have a hand for each round. </li>
    <li> The player will eventually need methods to do "poker stuff" - bet, fold, swap cards, etc...</li>
    </ul>
</ul>

We can still try to simplify things to make our project easier, in particular I'm going to leave out the idea of swapping cards out of a hand. So our poker came will be simplified to:
<ul>
<li> Deal 5 cards to each player. </li>
<li> Players can bet, call, or fold. </li>
<li> Determine winner and give prize. </li>
</ul>

As with our other simplifications we are leaving out some of the more complex bits to create a framework that is functional but incomplete. We can then go back to things like the scoring logic and fill them in after other parts are functional. 

### Player Actions

At this point we have the first real place where we need to capture and deal with a player decision, the decision of what to bet. Before we tackle this, we should look at how poker betting works in general (there are lots of variations, but this is more or less true):
<ul>
<li> Someone has to ante, or place a small forced bet to start the round. This rotates through players. </li>
<li> Players can then raise, call, or fold. </li>
<li> Each player must either match or raise the bet to stay in, or fold. </li>
<li> Once everyone has matched or folded, the round of betting is over and we do the next thing. </li>
</ul>

This framework is relatively simple, and I think that we can adapt something that works for this to a real game with multiple rounds of betting. One thing that we need to get to do this is an input from the player, what do they want to do with their hand. We can accommodate this by adding a method to the player class that will ask them what they want to do, and then return that decision. In our testing we can setup dummy actions for our players to take, but in the long run the method we build can be modified to work in a real game. For example, if our method accepts a decision of bet/call/fold, the logic of what we need to do in our game is done - accepting that decision can come from other places, such as another "real" player, a gamepad, the internet, etc... We don't need to conflate the "get a decision" and the "do the decision" parts of the code.

### Development Work

I'm going to make the new classes here, then paste them into the .py file when done. This isn't mandatory, I just find it easier to work directly here as we go, you can also just work in the .py file and import it as we did above.

In [7]:
class Player():

    def __init__(self, name, bank=1000):
        self.name = name
        self.hand = None
        self._bank = bank
    
    def setHand(self, hand):
        self.hand = hand

    def __str__(self):
        return self.name+" - : "+str(self._bank)+"\n"+str(self.hand)
    
    def __lt__(self, other):
        return self.hand < other.hand
    def __gt__(self, other):
        return self.hand > other.hand
    def __eq__(self, other):
        return self.hand == other.hand
    
    def checkHand(self):
        return self.hand.checkHand()
    
    def bet(self, amount):
        if amount > self._bank:
            raise ValueError("You don't have that much money")
        self._bank -= amount
        return amount
    
    def getBank(self):
        return self._bank
    def setBank(self, value):
        self._bank = value
    def addBank(self, value):
        self._bank += value
    def getHand(self):
        return self.hand
    def getName(self):
        return self.name

In [8]:
class FiveCardDraw():

    def __init__(self, num_players=4, num_cards=5, start_bank=1000, player_names=None):
        self.num_players = num_players
        self.num_cards = num_cards
        self.players = []

        for i in range(num_players):
            play_name = player_names[i]
            if play_name is None:
                self.players.append(Player("Player "+str(i), start_bank))
            else:
                self.players.append(Player(play_name, start_bank))
    
    def __str__(self):
        return_string = " "
        for i, hand in enumerate(self.players):
            return_string += self.players[i].name+":\n"+str(hand)+"\n"
        return return_string
    
    def calculateWinner(self, to_print=False):
        scores = []
        for hand in self.players:
            scores.append(hand.checkHand())
        if to_print:
            print(scores)
        return scores.index(max(scores))
    
    def playRound(self, shuffle=True, bets=[]):
        deck = Deck(shuffle=shuffle, populate=True)
        hands = list(map(Hand.deckToHand, deck.deal(num_hands=self.num_players, card_per_hand=self.num_cards)))
        pool = 0
        for i, hand in enumerate(hands):
            self.players[i].setHand(hand)
        
        if len(bets) > 0:
            for i, bet in enumerate(bets):
                val = self.players[i].bet(bet)
                pool += val

        winner = self.calculateWinner()
        self.players[winner].addBank(pool)
        return winner, hands[winner]
    
    def getPlayers(self):
        return self.players
    
    def swapCards(self, player, cards):
        oldHand = player.getHand()
        

## Test Harness

This scenario is starting to get complex enough that we will want to do some testing as we go. We can build a test harness, or a piece of code that runs some other piece of code for testing, that will do some runs of our game, so we can see the results easily. 

Building a test harness is a good idea for a few reasons:
<ul>
<li> It allows us to test our code as we go, without having to run the whole game. </li>
<li> It allows us to test our code in a controlled way, so we can see what happens when we do certain things. </li>
<li> It allows us easily do regression testing, or verify that we haven't broken something that was working before. </li>
</ul>

In ours, I want to cover the key things we are trying to make. This won't be totally comprehensive, but it will be representative. Some of the steps that I think I want to test are:
<ul>
<li> Creating a game. </li>
<li> Creating players and ensuring their names and banks work properly. </li>
<li> Dealing cards to players. </li>
<li> Evaluating winner. </li>
<li> Updating bankrolls. </li>
<li> Starting a new round from the ending state of the last one. (Our game keeps going, we shouldn't reset things like bankrolls). </li>
</ul>

Our game will need to do all of this stuff, so we can start with the tests that we need to pass, and work backwards. If all of this stuff works, we are likely good, up to the point that we are aiming for. As well, if something breaks as we make changes, we will notice that quickly, not after we've made dozens of other changes that can make it hard to find the problem.

To set this up, I can think of what steps are needed to do the work above, then code that. I can also think of what the output should be, and we can write checks to see if it matches the expected output. The checking part is obviously something that might not always be practical to do, but if it is possible it is something that can help us automate the testing process as much as possible. So the steps that I'll aim for are:
<ul>
<li> Make a new game with some named players and a provided bankroll. </li>
<li> Check that the players have the right names and bankrolls. </li>
<li> Deal cards to the players. </li>
<li> Check that the players have the right cards.* </li>
<li> Provide bets for the players. </li>
<li> Determine winners. </li>
<li> Change bankrolls. </li>
<li> Check that the bankrolls are correct. </li>
<li> Start a new round. </li>
<li> Check inital bankrolls. </li>
</ul>

This might change a bit as I go, but this is a good starting point. One thing to note is that we want to check the logic of who wins, and ensure that is correct so that we can test the bankrolls. For this test, I think that I might make the hands that each player gets be fixed, so we can know who should win. This will make it easier to test the new stuff, since we are already pretty confident that dealing different hands is ok from the testing we did outside the game.

We can do the actual tests in any way that we find convenient, one way is to use the assert statement. This statement will check if something is true, and if it is not, it will throw an error. This is a good way to check things that we expect to be true, and if they are not, we know that something is wrong. Using asserts can also integrate well with other testing and debugging frameworks, so it is good practice. There are several more elaborate testing frameworks that are more common in larger projects, but the idea is always the same - check and see if whatever we wrote passes whatever requirements we have.

#### Changes for Testing

As we build this we will likely encounter changes we need to make, even to the older parts of the code. Most notably, we may need some changes to get values from objects so we can do the testing. We'll probably also notice things that we need to add, such as the ability to add names to players in our game. That's normal to find, and we can make the changes as we go for smaller things. Some of the things we add might not be useful for "real" poker games, they might just be needed for us to test; that is fine, we can just not use them later. One of the core things that we likely want to change is when we have randomness - it is hard to test things that occur randomly, so we may want to have some type of switch to turn off randomness for testing.

### Player Test Function

Testing the player is relatively simple, since it doesn't do all that much. We want to test that each of the things that a player does works. We can go down the line and think about some things to test:
<ul><b>
<li> Creating a player with a name and bankroll. </li>
<li> Checking that the name in the player object matches what we set. </li>
<li> Checking that the bankroll in the player object matches what we set. </li>
<li> Give that player a hand. </li>
<li> Check that the hand matches what we gave it. </li>
<li> Give that player a new hand. </li>
<li> Check that the hand matches what we gave it. </li>
<li> Make a bet. </li>
<li> Check that the bet matches what we set. </li>
<li> Check that the bankroll matches what we set. </li>
</b></ul>

This list is pretty good for the functionality of the Player, so we can build unit tests to check all of these things. I think we'll have to check comparisons, but I'll do that in the game tests, for now we'll just test that one player works properly. Like most other things, what exactly we want to test for might grow or change as we build things and explore the details of our project more. For each one we want to check the functionality of the player class, so that will generally translate into:
<ul>
<li> Make some trial data up. </li>
<li> Use that data in the player class. </li>
<li> Check the resulting output of the player object, compare to what we know we want. </li>
</ul>

We can do this for each of the things we want to test, and we can do it in a function that we can call to run the tests. This is a good way to do things, since we can then run the tests whenever we want, and we can also add more tests as we go. If we are able to capture all possible cases, we can be near certain that our code works properly if it passes. If we are not able to capture every possible scenario, but are able to capture many or a representative sample, we can't be certain but we can be more confident. As well, if we notice issues, we can add tests to the list to address them as they occur. 

#### Player Test Implementation

The example I'm making here does the job, but it isn't the only way. On a larger project, we want to comply with the standards of the organization; for our smaller examples, focus on the idea of verifying that things work, and managing that testing in an organized way. For my example here, I'll structure some functions so they take in:
<ul>
<li> An object that we've made. </li>
<li> The data that we used as the input to the object, to test against. </li>
</ul>

I think this is a solid process to start with, and it allows us to easily test multiple scenarios by looping over different inputs. The assert statements here will throw an error up if they fail, and that error will have the text after the comma, so we can make the details of what failed clear.

<b>Note:</b> try changing something so it throws an error, and see what happens. This is a good way to see how the assert statements work.

## Exercise - Complete Test Harness for Player then Complete Player Class

We want to define the test assertions for a working player class here, then complete the player class so that it passes the tests. Try also running it with a different set of inputs - perhaps my limited test data didn't cover all possible outcomes. 

In [9]:
# Setup test data, we make this up
testName = "Bob"
testBank = 1000
testHand = Hand([Card("Hearts", "King"), Card("Hearts", "Queen"), Card("Hearts", "Jack"), Card("Hearts", "10"), Card("Hearts", "9")])
testHand2 = Hand([Card("Clubs", "King"), Card("Hearts", "Five"), Card("Hearts", "Jack"), Card("Diamonds", "10"), Card("Hearts", "9")])

testPlay = Player(testName, testBank)
testPlay.setHand(testHand)
testVal = 100

In [10]:
# Run the test
def testPlayer(player, name, hand1, bank, hand2, testVal=100):
    # Test that intial player state is what we expect. 
    # I checked the name, hand, and bank
    assert player.getName() == name, "Name failed"
    assert player.getHand() == hand1, "Hand check 1 failed"
    assert player.getBank() == bank, "Bank check 1 failed"

    # Change bank for testing. 
    # I set and added to the bank, then checked that it was correct
    player.setBank(testVal)
    player.addBank(testVal)
    assert player.getBank() == (2*testVal), "Bank check 2 failed"

    # Change hand for testing, check that the current hand is correct. 
    player.setHand(hand2)
    assert player.getHand() == hand2, "Hand check 2 failed"

    # Make a bet and ensure that the bank is correct after
    assert player.bet(testVal) == testVal, "Bet check 1 failed"
    assert player.getBank() == (testVal), "Bank check 3 failed"


In [11]:
try:
    testPlayer(testPlay, testName, testHand, testBank, testHand2, testVal)
except AssertionError as e:
    print("Failed testPlayer")
    print(e)

#### Game Test Function

We can also do some larger scale integration testing on the game, and use those tests to guide our development. 

The integration tests are aimed at the larger processes that are part of the game, not on specific functions. The scenarios will therefore be larger and less specific. I think that some things that are good to test are:
<ul><b>
<li> Can we make the game to its initial state, with players and bankrolls? How do we test that? </li>
<li> Can we play a hand, and how do we test that? </li>
<li> After a hand is played, can we start another from the ending state of the last one? </li>
</b></ul>

These tests don't target the specific functions in our game, we aren't testing things like comparing hands or dealing cards directly as we've done that when we created them. These tests are for entire processes, so they'll show us if things work, but likely won't pinpoint where errors occur. My intention here is to basically cover the mechanics of a game executing, without the personal interactions that would occur when you actually play. We'll eventually need to change how a bet is "submitted" to the game and a few other things, but we are working on keeping a limited working model as we go. 

<b>Note:</b> for this I don't care about taking in real bets, as you would in a game, I just need to be able to calculate the values correctly if those bets exist. For that reason I'm going to input the bets as an argument to playing a round. This will likely need to change when we refactor, but there will probably be several changes that we'll make when we think about the actual mechanics of playing a game, for now that does the job to test the logic. 


## Exercise - Complete Test Harness and Game Class

Complete the tests for the game class, then complete the game class so that it passes the tests.

In [12]:
players = ["Bob", "Alice", "Charlie", "David"]
initialBank = 1000
bets1 = [50, 50, 50, 50]
bets2 = [100, 100, 100, 100]
target_banks1 = [1150, 950, 950, 950]
target_banks2 = [1450, 850, 850, 850]

game1 = FiveCardDraw(player_names=players)

In [13]:

def testGame(players, initialBank, target_banks1, target_banks2, testGame, bets1, bets2):
    # Test that initial game state matches expectations. 
    # I checked that the names match what we set and the bank matches what we set
    game_players = testGame.getPlayers()
    for i, player in enumerate(game_players):
        assert player.getName() == players[i], "Name check failed "+str(player.getName())+" "+str(players[i])
        assert player.getBank() == initialBank, "Bank check failed "+str(player.getBank())
    
    # Play a round and check the state after to see that it is correct. 
    # I checked that the winner is correct (think about the game play, and what might need to change to make it testable)
    # I also checked that the resulting bank values are correct
    playWin, winHand = testGame.playRound(shuffle=False, bets=bets1)
    #print("Winner: "+str(playWin))
    #print(winHand)
    assert playWin == 0, "Winner check failed "+str(playWin)
    for i, player in enumerate(game_players):
        assert player.getBank() == target_banks1[i], "Bank check failed "+str(player.getBank())+" "+str(target_banks1[i])+" "+str(player.getName())
    
    # Play another round and check the state after to see that it is correct. 
    # I checked that the winner is correct (think about the game play, and what might need to change to make it testable)
    # I also checked that the resulting bank values are correct, and reflect both rounds. 
    playWin2, winHand2 = testGame.playRound(shuffle=False, bets=bets2)
    #print("Winner: "+str(playWin2))
    #print(winHand2)
    assert playWin2 == 0, "Winner check failed "+str(playWin2)
    for i, player in enumerate(game_players):
        assert player.getBank() == target_banks2[i], "Bank check failed "+str(player.getBank())+" "+str(target_banks2[i])+" "+str(player.getName())

In [14]:
try:
    testGame(players, initialBank, target_banks1, target_banks2, game1, bets1, bets2)
except AssertionError as e:
    print("Failed testGame")
    print(e)

### Wrapping Up

Once I'm happy with the code we've made here I'll just copy and paste into the .py file and then work from that base going forward. So for the next round of changes we'll import that file as we did here, and make another set of additions or modifications. 

## Big Note

All of this stuff is somewhat open-ended and vague, that's normal. We often don't know exactly what a final solution will look like or how we'll implement it, but we know part of what we need and can work towards a solution. Navigating this abmiguity is a key skill to develop, even if it is frustrating at times.

### Extension

Our game is getting closer and close to being functional, some things that we can add, and that we will look at in the future, are:
<ul>
<li> Real scoring and comparison logic. </li>
<li> Multiple rounds of betting. </li>
<li><b> Card exchanges - there are some stubs of functionality in the .py solution file that are not fully tested. Expanding and building a test harness for these is a good exercise. </b> </li>
</ul>